In [1]:
import concurrent.futures
import requests
import re 
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#2001
base_url = 'https://wybory2001.pkw.gov.pl/'

main_page_url = base_url+'sjg2_k.html'
page = requests.get(main_page_url)

In [3]:
# here, we fetch the content from the url, using the requests library
page_content = BeautifulSoup(page.content, "html.parser")
#we use the html parser to parse the url content and store it in a variable.textContent = []
paragraphs = page_content.findAll('a')
wojew = []
for p in paragraphs:
    if len(re.findall(r'<a href="([0-9]*\/sjg2_w.html)">(.*)</a>', str(p))) == 1:
        wojew.append( re.findall(r'<a href="([0-9]*\/sjg2_w.html)">(.*)</a>', str(p))[0])
# In my use case, I want to store the speech data I mentioned earlier.  so in this example, I loop through the paragraphs, and push them into an array so that I can manipulate and do fun stuff with the data.

In [4]:
#.encode("iso-8859-1").decode('utf-8')

In [5]:
path_0 = '/home/marek/Dokumenty/MGR_CLEAN/wyniki_wyborow/2001/'

for w in wojew:
    print(w)
print(len(wojew))
    
df_0 = pd.DataFrame(wojew, columns =['Komitet', 'Glosy']) 
df_0.Glosy = df_0.Glosy.str.replace('\s', '')

('020000/sjg2_w.html', 'dolno¶l±skie')
('040000/sjg2_w.html', 'kujawsko-pomorskie')
('060000/sjg2_w.html', 'lubelskie')
('080000/sjg2_w.html', 'lubuskie')
('100000/sjg2_w.html', '³ódzkie')
('120000/sjg2_w.html', 'ma³opolskie')
('140000/sjg2_w.html', 'mazowieckie')
('160000/sjg2_w.html', 'opolskie')
('180000/sjg2_w.html', 'podkarpackie')
('200000/sjg2_w.html', 'podlaskie')
('220000/sjg2_w.html', 'pomorskie')
('240000/sjg2_w.html', '¶l±skie')
('260000/sjg2_w.html', '¶wiêtokrzyskie')
('280000/sjg2_w.html', 'warmiñsko-mazurskie')
('300000/sjg2_w.html', 'wielkopolskie')
('320000/sjg2_w.html', 'zachodniopomorskie')
16


In [88]:
df_0

,Komitet,Glosy
0,020000/sjg2_w.html,dolno¶l±skie
1,040000/sjg2_w.html,kujawsko-pomorskie
2,060000/sjg2_w.html,lubelskie
3,080000/sjg2_w.html,lubuskie
4,100000/sjg2_w.html,³ódzkie
5,120000/sjg2_w.html,ma³opolskie
6,140000/sjg2_w.html,mazowieckie
7,160000/sjg2_w.html,opolskie
8,180000/sjg2_w.html,podkarpackie
9,200000/sjg2_w.html,podlaskie
